### Learning Connecting SQL with Python to Looker Studio (Free) via Google Sheet 

**Goal:**
- Extract data from MySQL
- Clean and validate data
- Aggregate into analytics-ready KPIs
- Prepare output for Google Sheets to Looker Studio

In [29]:
import mysql.connector # we use this to connect with the sql
import pandas as pd

conn = mysql.connector.connect(
    host="localhost",
    user="analytics_user",
    password="Analytics@123",
    database="restaurant_db",
    port=3306
)

cursor = conn.cursor() # we use this execuate manually

print("Connected to MySQL")

Connected to MySQL


In [30]:
cursor.execute("SELECT * FROM order_details;")

rows = cursor.fetchall() # to fetch rows only
columns = [col[0] for col in cursor.description]  


# Let's understand what we have done here :)
"""
>>> cursor.execute
        This prep. both the data and metadata
        
>>> Now what is cursor.description?
    - This helps us to get the col^s metadata only

>>> What Does cursor.description Look Like?
    - It's just a list of tuples & each tuple represents one col^m
    - example: cursor.description
[
  ('order_details_id', 3, None, None, None, None, None),
  ('order_id', 3, None, None, None, None, None),
  ('order_date', 10, None, None, None, None, None),
  ('order_time', 11, None, None, None, None, None),
  ('item_id', 3, None, None, None, None, None)
]

>>> And each tuple contains 7 elements which is

                (
                  name,          ← index 0
                  type_code,     ← index 1
                  display_size,  ← index 2
                  internal_size, ← index 3
                  precision,     ← index 4
                  scale,         ← index 5
                  null_ok        ← index 6
                )

                
        # Which means:
        
            Col[0] - Col Name
            Col[1] - Data Type Code - SQL internal code and not human friendly
            Col[2] - Other metadata - low level database info.
                And most importantly pandas doesnt need this.
                

Now I guess you have got an idea why we have choosen Col[0]

    For each column metadata tuple, extract the column name.
        So that the output becomes
            ['order_details_id', 'order_id', 'order_date', 'order_time', 'item_id']
"""


order_details_df = pd.DataFrame(rows, columns=columns)

In [31]:
order_details_df.head()

,order_details_id,order_id,order_date,order_time,item_id
0,1,1,2023-01-01,0 days 11:38:36,109.0
1,2,2,2023-01-01,0 days 11:57:40,108.0
2,3,2,2023-01-01,0 days 11:57:40,124.0
3,4,2,2023-01-01,0 days 11:57:40,117.0
4,5,2,2023-01-01,0 days 11:57:40,129.0


In [32]:
cursor.execute("SELECT * FROM menu_items;")

rows = cursor.fetchall()
columns = [col[0] for col in cursor.description]

menu_items_df = pd.DataFrame(rows, columns=columns)

In [33]:
menu_items_df.head()

,menu_item_id,item_name,category,price
0,101,Hamburger,American,12.95
1,102,Cheeseburger,American,13.95
2,103,Hot Dog,American,9.00
3,104,Veggie Burger,American,10.50
4,105,Mac & Cheese,American,7.00


In [34]:
query = """
SELECT
    od.order_id,
    od.order_date,
    od.order_time,
    mi.category,
    mi.price
FROM order_details od
JOIN menu_items mi
  ON od.item_id = mi.menu_item_id;
"""

raw_df = pd.read_sql(query, conn)

/var/folders/wy/bt30t8cs73vf72snh3hxbkr80000gn/T/ipykernel_11167/3994067296.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_df = pd.read_sql(query, conn)


In [35]:
raw_df.head()

,order_id,order_date,order_time,category,price
0,1,2023-01-01,0 days 11:38:36,Asian,17.95
1,2,2023-01-01,0 days 11:57:40,Asian,14.50
2,2,2023-01-01,0 days 11:57:40,Italian,14.50
3,2,2023-01-01,0 days 11:57:40,Mexican,12.95
4,2,2023-01-01,0 days 11:57:40,Italian,15.50


In [36]:
# Convert date & time to proper formats
raw_df["order_date"] = pd.to_datetime(raw_df["order_date"], errors="coerce")
raw_df["order_time"] = pd.to_timedelta(raw_df["order_time"], errors="coerce")

In [37]:
# Remove invalid rows
raw_df = raw_df.dropna(subset=["order_date", "order_time", "order_id", "price"])

In [38]:
# Handle category (to be renamed later)
raw_df["category"] = raw_df["category"].fillna("Unknown")

In [39]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12097 entries, 0 to 12096
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   order_id    12097 non-null  int64          
 1   order_date  12097 non-null  datetime64[ns] 
 2   order_time  12097 non-null  timedelta64[ns]
 3   category    12097 non-null  object         
 4   price       12097 non-null  float64        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1), timedelta64[ns](1)
memory usage: 472.7+ KB


In [40]:
raw_df.isnull().sum()

order_id      0
order_date    0
order_time    0
category      0
price         0
dtype: int64

In [41]:
daily_kpis = (raw_df.groupby(raw_df["order_date"].dt.date).agg(orders=("order_id", "nunique"),revenue=("price", "sum")).reset_index())

In [42]:
daily_kpis["aov"] = daily_kpis["revenue"] / daily_kpis["orders"]

In [43]:
daily_kpis.rename(columns={"order_date": "date"}, inplace=True)

In [44]:
daily_kpis.head()

,date,orders,revenue,aov
0,2023-01-01,68,2091.60,30.758824
1,2023-01-02,66,1994.70,30.222727
2,2023-01-03,64,1983.70,30.995313
3,2023-01-04,52,1356.85,26.093269
4,2023-01-05,54,1589.85,29.441667


In [45]:
from email.message import EmailMessage
import smtplib
import os
from datetime import datetime

APP_PASSWORD = os.getenv("GMAIL_APP_PASSWORD")

now = datetime.now().strftime("%Y-%m-%d %H:%M")

subject = f"KPI Report | {now}"

email_body = f"""
Hi,

Please find attached the KPI report.

Generated at: {now}

Regards,
Automated Analytics Pipeline
"""

msg = EmailMessage()
msg["Subject"] = subject
msg["From"] = "luxevistahub@gmail.com"
msg["To"] = "career.pranab@gmail.com"

msg.set_content(email_body)

msg.add_attachment(
    csv_data.encode("utf-8"),
    maintype="text",
    subtype="csv",
    filename="daily_kpis.csv"
)

server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
server.login("luxevistahub@gmail.com", APP_PASSWORD)

"""
#Login with the sender id
server.login(
    "luxevistahub@gmail.com",
    "abcd abcd abcd abcd".replace(" ", "")
    
    ---Incase you are using this for your person staff and app password doesn't matter (unsafe way) ---
        In such scenerio you can keep your app password public
    
"""
server.send_message(msg)
server.quit()

print("Email sent successfully with CSV attachment")

Email sent successfully with CSV attachment
